# Run workflow with c3s-cica-atlas v1 data

**Rooki** calls climate data operations on the **rook** processing service.

In [1]:
import os
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'

from rooki import rooki
from rooki import operators as ops

## workflow: subset ERA5


In [2]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.tx.ERA5.mon']
)

wf = ops.Subset(
    ds, 
    time="2000/2000",
    time_components="month:jan,feb,mar",
    area='0.,49.,10.,65',
)

resp = wf.orchestrate()
resp.ok

True

In [3]:
resp.download_urls()

['http://rook4.cloud.dkrz.de:80/outputs/rook/12dea2d0-bed3-11ee-af1d-fa163ed6c06f/tx_ERA5_no-expt_mon_20000101-20000301.nc']

In [4]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:    (lat: 65, bnds: 2, lon: 41, time: 3)
Coordinates:
  * lat        (lat) float64 49.0 49.25 49.5 49.75 ... 64.25 64.5 64.75 65.0
  * lon        (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 9.0 9.25 9.5 9.75 10.0
  * time       (time) datetime64[ns] 2000-01-01 2000-02-01 2000-03-01
    height2m   float64 ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    time_bnds  (time, bnds) datetime64[ns] ...
    tx         (time, lat, lon) float32 ...
    crs        int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         179.875
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2023-10-23 11:32:52.726326+02:00
    tracking_id:                35730460-414f-41df-988b-674b3e9def74
    project_id:                 c3s-cica-atlas

## workflow: subset ERA5-Land

In [5]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.fd.ERA5-Land.mon']
)

wf = ops.Subset(
    ds, 
    time="2000/2000",
    time_components="month:jan,feb,mar",
    area='0.,49.,10.,65',
)

resp = wf.orchestrate()
resp.ok

True

In [6]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:      (lat: 160, bnds: 2, lon: 100, time: 3)
Coordinates:
  * lat          (lat) float64 49.1 49.2 49.3 49.4 49.5 ... 64.7 64.8 64.9 65.0
  * lon          (lon) float64 0.01099 0.111 0.211 0.311 ... 9.712 9.812 9.912
  * time         (time) datetime64[ns] 2000-01-01 2000-02-01 2000-03-01
    threshold0c  float64 ...
    height2m     float64 ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds     (lat, bnds) float64 ...
    lon_bnds     (lon, bnds) float64 ...
    time_bnds    (time, bnds) datetime64[ns] ...
    fd           (time, lat, lon) float32 ...
    crs          int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         179.9719696044922
    geospatial_lon_resolution:  0.100006103515625
    geospatial_lon_units:       degrees_east
    date_created:               2023-11-03 11:22:46.830730+01:00
    tracking_id:                888a50fd-716c-4755-9c0f-2d029a53ffb4
    project_id:                 c3s-cica-atlas

## workflow: subset cmip5

In [7]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.pr.CMIP5.historical.mon']
)

wf = ops.Subset(
    ds, 
    time="2000/2000",
    time_components="month:jan,feb,mar",
    area='0.,49.,10.,65',
)

resp = wf.orchestrate()
resp.ok

True

In [8]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:          (lat: 9, bnds: 2, lon: 5, time: 3, member: 79)
Coordinates:
  * lat              (lat) float64 49.0 51.0 53.0 55.0 57.0 59.0 61.0 63.0 65.0
  * lon              (lon) float64 1.0 3.0 5.0 7.0 9.0
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 2000-03-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    pr               (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  2.0
    geospatial_lon_units:       degrees_east
    date_created:               2023-10-23 10:18:38.576946+02:00
    tracking_id:                f186e3e2-8523-4785-abd4-975147ec8240
    project_id:                 c3s-cica-atlas

## workflow: subset cmip6

In [9]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.fd.CMIP6.historical.mon']
)

wf = ops.Subset(
    ds, 
    time="2000/2000",
    time_components="month:jan,feb,mar",
    area='0.,49.,10.,65',
)

resp = wf.orchestrate()
resp.ok

True

In [10]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:          (lat: 16, bnds: 2, lon: 10, time: 3, member: 26)
Coordinates:
  * lat              (lat) float64 49.5 50.5 51.5 52.5 ... 61.5 62.5 63.5 64.5
  * lon              (lon) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 2000-03-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    threshold0c      float64 ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    fd               (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2023-09-28 13:44:24.110010+02:00
    tracking_id:                363128f3-0c81-4b08-9dc0-2b9fca993bd6
    project_id:                 c3s-cica-atlas

## workflow: subset ORAS5

In [11]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.sst.ORAS5.mon']
)

wf = ops.Subset(
    ds, 
    time="2000/2000",
    time_components="month:jan,feb,mar",
    area='0.,49.,10.,65',
)

resp = wf.orchestrate()
resp.ok

True

In [12]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:    (lat: 64, bnds: 2, lon: 40, time: 3)
Coordinates:
  * lat        (lat) float64 49.12 49.38 49.62 49.88 ... 64.12 64.38 64.62 64.88
  * lon        (lon) float64 0.125 0.375 0.625 0.875 ... 9.125 9.375 9.625 9.875
  * time       (time) datetime64[ns] 2000-01-01 2000-02-01 2000-03-01
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    time_bnds  (time, bnds) datetime64[ns] ...
    sst        (time, lat, lon) float32 ...
    crs        int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2023-10-24 11:01:24.009195+02:00
    tracking_id:                56ab2036-84aa-4f05-9e45-0bc540e4640a
    project_id:                 c3s-cica-atlas

## workflow: subset cordex eur-11

In [13]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.fd.CORDEX-EUR-11.historical.mon']
)

wf = ops.Subset(
    ds, 
    time="2000/2000",
    time_components="month:jan,feb,mar",
)

resp = wf.orchestrate()
resp.ok

True

In [14]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:          (lat: 408, bnds: 2, lon: 881, time: 3, member: 72)
Coordinates:
  * lat              (lat) float64 21.81 21.94 22.06 22.19 ... 72.44 72.56 72.69
  * lon              (lon) float64 -44.81 -44.69 -44.56 ... 64.94 65.06 65.19
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 2000-03-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    rcm_institution  (member) object ...
    rcm_model        (member) object ...
    rcm_variant      (member) object ...
    threshold0c      float64 ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    fd               (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         65.25
    geospatial_lon_resolution:  0.125
    geospatial_lon_units:       degrees_east
    date_created:               2023-12-04 15:45:53.359311+01:00
    tracking_id:                69e7a46d-7266-4705-be79-a106f133f94d
    project_id:                 c3s-cica-atlas

## workflow: subset cordex core

In [15]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.rlds.CORDEX-CORE.historical.mon']
)

wf = ops.Subset(
    ds, 
    time="2000/2000",
    time_components="month:jan,feb,mar",
)

resp = wf.orchestrate()
resp.ok

True

In [16]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:          (lat: 720, bnds: 2, lon: 1440, time: 3, member: 9)
Coordinates:
  * lat              (lat) float64 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon              (lon) float64 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 2000-03-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    rcm_model        (member) object ...
    rlds             (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2023-12-04 17:51:41.366602+01:00
    tracking_id:                29b12dbd-9dea-4294-9e8f-16a9168ac9f1
    project_id:                 c3s-cica-atlas

## workflow: complete time range

In [24]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.rlds.CORDEX-CORE.historical.mon']
)

wf = ops.Subset(
    ds, 
    # time="2000/2000",
)

resp = wf.orchestrate()
resp.ok

True

In [18]:
resp.download_urls()

['https://data.mips.copernicus-climate.eu/thredds/fileServer/esg_c3s-cica-atlas/CORDEX-CORE/historical/rlds_CORDEX-CORE_historical_mon_197001-200512.nc']

## workflow: most of the time range

In [26]:
ds = ops.Input(
    'ds', ['c3s-cica-atlas.rlds.CORDEX-CORE.historical.mon']
)

wf = ops.Subset(
    ds, 
    time="1979/2000",
)

resp = wf.orchestrate()
resp.ok

True

In [27]:
resp.size_in_mb

821.2735538482666

In [28]:
resp.download_urls()

['http://rook4.cloud.dkrz.de:80/outputs/rook/33ee56f0-bed8-11ee-9f5a-fa163ed6c06f/rlds_CORDEX-CORE_historical_mon_19790101-19871101.nc',
 'http://rook4.cloud.dkrz.de:80/outputs/rook/33ee6f46-bed8-11ee-9f5a-fa163ed6c06f/rlds_CORDEX-CORE_historical_mon_19871201-19961001.nc',
 'http://rook4.cloud.dkrz.de:80/outputs/rook/33ee7d24-bed8-11ee-9f5a-fa163ed6c06f/rlds_CORDEX-CORE_historical_mon_19961101-20001201.nc']